In [1]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField, TimestampType, DateType, ArrayType

In [2]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [3]:
sqlContext = SQLContext(sc)

In [4]:

def loadData(loc):
    df = sqlContext.read.format("csv") \
    .option("header", "true") \
    .option("timestampFormat", "MM/dd/yyyy HH:mm:SS") \
    .option("inferSchema", "true") \
    .load(loc)
    
    return df

In [6]:
bucket = 'gs://project_ggallagher6/' #201410-citibike-tripdata.csv'

def writeOutput(df, loc):
    df.repartition(1).write.option("header", "true").csv(loc)

In [7]:
df = loadData(bucket+'cleaned/stations.csv')

In [15]:
directionLookup = {'E ': 'East ', 'W ': 'West ', 'S ': 'South ', 'N ': 'North '}
streetLookup = {' Pl ': ' Place ', ' St ': ' Street ', ' Ln ': ' Lane '}
#remember to scrape off last letter if space
#be careful of lettered avenues - handle manually
#bob = df.withColumn('test', split(df['station_name'], '& ').getItem(0)) \
#.withColumn('test2', split(df['station_name'], '& ').getItem(1))
#bob.show()
#tim = bob.withColumn('test3', when(df['test']))

out = df.withColumn('street_1', rtrim(split(df['station_name'], '& ').getItem(0))) \
.withColumn('street_2', split(df['station_name'], '& ').getItem(1))
out = out.select('station_id', 'street_1', 'street_2', 'station_latitude', 'station_longitude')

In [17]:
out.show(4)
writeOutput(out, bucket+'test/')

+----------+--------+-----------+----------------+-----------------+
|station_id|street_1|   street_2|station_latitude|station_longitude|
+----------+--------+-----------+----------------+-----------------+
|      2001|   7 Ave|Farragut St|     40.69892054|     -73.97332996|
|       297| E 15 St|      3 Ave|       40.734232|       -73.986923|
|       476| E 31 St|      3 Ave|     40.74394314|     -73.97966069|
|       388| W 26 St|     10 Ave|    40.749717753|    -74.002950346|
+----------+--------+-----------+----------------+-----------------+
only showing top 4 rows

